In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [3]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_18992\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [4]:
COLUMNAS_SERVICIO_TRASLADO = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}

TIPOS_DE_CAMAS = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [14]:
# Convierte los servicios y las fechas en formato largo
servicios_egreso_long = df_torax.melt(
    id_vars=["id_egreso", "DIAGNOSTICO1", "ANIO_EGRESO"],
    value_vars=COLUMNAS_SERVICIO_TRASLADO,
    var_name="tipo_servicio",
    value_name="servicio",
)

fechas_egreso_long = df_torax.melt(
    id_vars=["id_egreso", "DIAGNOSTICO1", "ANIO_EGRESO"],
    value_vars=COLUMNAS_FECHA_TRASLADO,
    var_name="tipo_fecha",
    value_name="fecha",
)

In [15]:
# Une los servicios y fechas respectivas para que esten en formato contiguo
recorrido_egreso = pd.merge(
    left=fechas_egreso_long,
    right=servicios_egreso_long,
    how="inner",
    left_index=True,
    right_index=True,
)

# Indica el tipo de ordenamiento del tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
recorrido_egreso["tipo_servicio"] = pd.Categorical(
    recorrido_egreso["tipo_servicio"],
    categories=COLUMNAS_SERVICIO_TRASLADO.keys(),
    ordered=True,
)

# Ordena cada egreso segun su tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
recorrido_egreso = recorrido_egreso.sort_values(["id_egreso_x", "tipo_servicio"])

# Elimina filas sin ninguna fecha ingresada
recorrido_egreso = recorrido_egreso.dropna()

# Resetea los indices para que sean contiguos
recorrido_egreso = recorrido_egreso.reset_index(drop=True)

# Imputa fechas con formato de fechas
recorrido_egreso["fecha"] = pd.to_datetime(recorrido_egreso["fecha"])

In [16]:
# Calcula el tiempo de estancia por servicio
recorrido_egreso["tiempo_de_estancia"] = (
    recorrido_egreso.groupby("id_egreso_x")["fecha"].diff().shift(-1)
)

# Identifica las fechas mal imputadas, donde el tiempo de estancia es negativo
indices_egresos_con_tiempos_negativos = (
    recorrido_egreso[recorrido_egreso["tiempo_de_estancia"] < np.timedelta64(0)].index + 1
)

# Modifica las fechas erroneas, agregando 1 anio a la fecha
recorrido_egreso.loc[indices_egresos_con_tiempos_negativos, "fecha"] = recorrido_egreso.loc[
    indices_egresos_con_tiempos_negativos, "fecha"
] + pd.offsets.DateOffset(years=1)

# Recalcula el tiempo de estancia por servicio
recorrido_egreso["tiempo_de_estancia"] = (
    recorrido_egreso.groupby("id_egreso_x")["fecha"].diff().shift(-1)
)

# Identifica las fechas mal imputadas, donde el tiempo de estancia es negativo
indices_egresos_con_tiempos_0 = (
    recorrido_egreso[recorrido_egreso["tiempo_de_estancia"] == np.timedelta64(0)].index
)

# Modifica las fechas donde el tiempo es 0, imputandolos como 1 dia
recorrido_egreso.loc[indices_egresos_con_tiempos_0, "tiempo_de_estancia"] = recorrido_egreso.loc[
    indices_egresos_con_tiempos_0, "tiempo_de_estancia"
] + pd.Timedelta(days=1)

# Modifica el nombre del tipo de camas
recorrido_egreso["servicio"] = recorrido_egreso["servicio"].replace(TIPOS_DE_CAMAS)

In [17]:
recorrido_egreso

,id_egreso_x,DIAGNOSTICO1_x,ANIO_EGRESO_x,tipo_fecha,fecha,id_egreso_y,DIAGNOSTICO1_y,ANIO_EGRESO_y,tipo_servicio,servicio,tiempo_de_estancia
0,0,I25.1,2019.0,FECHA_INGRESO,2019-01-14,0,I25.1,2019.0,SERVICIOINGRESO,Medias,3 days
1,0,I25.1,2019.0,FECHAALTA,2019-01-17,0,I25.1,2019.0,SERVICIOALTA,Medias,NaT
2,1,Z51.1,2019.0,FECHA_INGRESO,2019-08-26,1,Z51.1,2019.0,SERVICIOINGRESO,Medias,1 days
3,1,Z51.1,2019.0,FECHAALTA,2019-08-26,1,Z51.1,2019.0,SERVICIOALTA,Medias,NaT
4,2,Z51.1,2019.0,FECHA_INGRESO,2019-01-09,2,Z51.1,2019.0,SERVICIOINGRESO,Medias,1 days
...,...,...,...,...,...,...,...,...,...,...,...
37403,13465,I25.1,2021.0,FECHATRASLADO2,2021-10-29,13465,I25.1,2021.0,SERVICIOTRASLADO2,UCI,11 days
37404,13465,I25.1,2021.0,FECHATRASLADO3,2021-11-09,13465,I25.1,2021.0,SERVICIOTRASLADO3,Medias,1 days
37405,13465,I25.1,2021.0,FECHATRASLADO4,2021-11-10,13465,I25.1,2021.0,SERVICIOTRASLADO4,UCI,2 days
37406,13465,I25.1,2021.0,FECHATRASLADO5,2021-11-12,13465,I25.1,2021.0,SERVICIOTRASLADO5,UTI,4 days


In [19]:
resumen_diagnosticos = (
    recorrido_egreso.groupby(["ANIO_EGRESO_x", "DIAGNOSTICO1_x", "servicio"])["tiempo_de_estancia"].sum().unstack()
)

In [21]:
resumen_diagnosticos.reset_index()

servicio,ANIO_EGRESO_x,DIAGNOSTICO1_x,AMBULATORIO,Basicos,Medias,UCI,UTI
0,2019.0,A08.4,NaT,NaT,3 days,2 days,NaT
1,2019.0,A09.9,NaT,NaT,16 days,3 days,30 days
2,2019.0,A15.0,NaT,NaT,169 days,1 days,48 days
3,2019.0,A15.2,NaT,NaT,74 days,NaT,39 days
4,2019.0,A15.3,NaT,NaT,74 days,NaT,26 days
...,...,...,...,...,...,...,...
1062,2021.0,Z45.0,NaT,NaT,139 days,1 days,37 days
1063,2021.0,Z47.0,NaT,NaT,9 days,NaT,6 days
1064,2021.0,Z51.1,NaT,NaT,12 days,NaT,NaT
1065,2021.0,Z51.2,NaT,NaT,5 days,NaT,1 days
